In [1]:
# setup disply parameters
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
from matplotlib.ticker import StrMethodFormatter
float_formatter = StrMethodFormatter('{x:0.03f}')
%pylab inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%config InlineBackend.figure_formats = ['retina', 'pdf', 'png']

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
pylab.rcParams['figure.figsize'] = (18, 6)

Populating the interactive namespace from numpy and matplotlib


/Users/yasser/.local/share/virtualenvs/negmas-bNSbQ2CL/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


<style>.container { width:95% !important; }</style>

## Develop a factory manager (agent) for the SCM world

This tutorial describes how to develop an agent for the SCM world, test it, and submit it to the ANAC 2019 SCM league.

The first step is to install negmas

```bash
pip install negmas
```
Once you have this library installed, you can start developing your factory manager:

You are told that you cannot instantiate your newly created class as did not implement the abstract (required) method ```on_negotiation_request```. That is actually the ONLY abstract method you need to implement.

In [8]:
class MyFactoryManager(FactoryManager):
    """My factory manager"""

As the [documentation](http://www.yasserm.com/negmas/negmas.apps.scml.html?highlight=on_negotiation_request#negmas.apps.scml.SCMLAgent.on_negotiation_request) states, this function is called whenever your factory manager receives a request from another agent to negotiate. You can either return `None` if you do not want to accept this negotiation or create  a `Negotiator` that represents your agent in it.


Your do-nothing agent is almost ready. Let's try it:

The property `stats` in `World` gives you several statistics about the world for every time-step of the simulation.

Let's check the contracts of this world:

Let's try to run a tournament with this do-nothing agent against the built-in greedy agent:

In [10]:
from negmas.apps.scml.utils import anac2019_tournament
from negmas.apps.scml import GreedyFactoryManager

n_completed = 0


def print_progress(_) -> None:
    global n_completed
    print(f'{n_completed} worlds completed')
    n_completed += 1


def print_world_progress(world) -> None:
    print(f'{world.current_step}  of {world.n_steps} steps completed')    


results = anac2019_tournament(competitors=(MyFactoryManager, GreedyFactoryManager), agent_names_reveal_type=True
                              , n_runs=4, n_steps=10, parallelism='serial', n_default_per_level=5
                      , n_factories_per_level=11, n_intermediate=3
                      , default_signing_delay=0, max_storage=1000000
                      , max_insurance_premium=100, n_retrials=5
                      , tournament_progress_callback=print_progress
                      , world_progress_callback=print_world_progress)

1  of 10 steps completed
2  of 10 steps completed
3  of 10 steps completed
4  of 10 steps completed
5  of 10 steps completed
6  of 10 steps completed
7  of 10 steps completed
8  of 10 steps completed
9  of 10 steps completed
10  of 10 steps completed
0 worlds completed
1 worlds completed
Traceback (most recent call last):
  File "/Users/yasser/code/projects/negmas/negmas/situated.py", line 1807, in tournament
    scores.append(_process_world_run(score_, tournament_name=name, dir_name=str(dir_name)))
  File "/Users/yasser/code/projects/negmas/negmas/situated.py", line 1683, in _process_world_run
    , columns=['agent_name', 'score', 'agent_type', 'log_file_names', 'world_name', 'stats_folder'])
  File "/Users/yasser/.local/share/virtualenvs/negmas-bNSbQ2CL/lib/python3.6/site-packages/pandas/core/frame.py", line 435, in __init__
    arrays, columns = to_arrays(data, columns, dtype=dtype)
  File "/Users/yasser/.local/share/virtualenvs/negmas-bNSbQ2CL/lib/python3.6/site-packages/pandas/cor

ValueError: No objects to concatenate

You can see the scores that each individual factory manager got:

In [ ]:
results.scores.head()

You can also check the total scores for each factory manager type:

In [ ]:
results.total_scores

If you want, you can check if these differences are statistically significant using a t-test:

In [ ]:
results.ttest

So the greedy factory manager is better than the do-nothing factory manager getting a total score of 2878 compared with only 367 for the do-nothing factory manager. Moreover, this difference is statistically significant as the p-value is 0.041 < 0.05.

Or, let's just find out who was the winner:

In [ ]:
results.winner

and what was its to score:

In [ ]:
results.winner_score

Now, you completed the development of your factory manager, tested it by running it in worlds and tournaments, what exactly should you do to participate in the SCM league @ ANAC 2019:

You need to submit the following items:

- Names of all members of the team with their affiliations and email addresses
- Either a single python file with the whole implementation of your agent with any supporting code or a zip file with a single folder containing your code. In the later case, you will need to indicate the class name of your factory manager. Any factory manager names are accepted except (Insurance, Bank, MFactoryManager, CFactoryManager).
- A 2-pages academic report about your factory manager. Please check the submission website for details about this report.

That is it folks!
You can now start developing your own factory manager. Have fun.